In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler # numeric scaling
from sklearn.metrics import accuracy_score # Metric for the competition
from catboost import CatBoostClassifier # Our model of choice
from sklearn.model_selection import GridSearchCV # Model tuning

In [2]:
df_train = pd.read_csv("train.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_test = pd.read_csv("test.csv")
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [6]:
print("Null values in training set: ", df_train.isnull().sum().sum())
print("Null values in testing set: ", df_test.isnull().sum().sum())

Null values in training set:  866
Null values in testing set:  414


In [7]:
replaceWithNone = [
    'Cabin'
]

for col in replaceWithNone:
    df_train[col]=df_train[col].fillna('None')
    df_test[col]=df_test[col].fillna('None')

replaceWithMode = [
    'Embarked'
]

for col in replaceWithMode:
    df_train[col]=df_train[col].fillna(df_train[col].mode()[0])
    df_test[col]=df_test[col].fillna(df_test[col].mode()[0])

replaceWithMean = [
    'Age', 'Fare'
]

for col in replaceWithMean:
    df_train[col]=df_train[col].fillna(df_train[col].mean())
    df_test[col]=df_test[col].fillna(df_test[col].mean())

In [8]:
print("Null values in training set: ", df_train.isnull().sum().sum())
print("Null values in testing set: ", df_test.isnull().sum().sum())

Null values in training set:  0
Null values in testing set:  0


In [9]:
X_train = df_train.drop(["PassengerId", "Survived"], axis=1)
y_train = df_train["Survived"]
X_test = df_test.drop("PassengerId", axis=1)

In [10]:
numeric_features = X_train.select_dtypes(exclude=["object"]).columns
categorical_features = df_train.select_dtypes(include=["object"]).columns

In [11]:
scaler = StandardScaler()
scaler.fit(X_train[numeric_features])

X_train_scaled = X_train
X_train_scaled[numeric_features] = scaler.transform(X_train[numeric_features])
X_test_scaled = X_test
X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])

In [12]:
lr = 0.1
dep = 2
#################### UNCOMMENT TO RUN GRID SEARCH ####################
# model = CatBoostClassifier(iterations=500, cat_features=categorical_features, loss_function="CrossEntropy", logging_level="Silent")

# # Change these values to tweak the model
# grid = {"learning_rate": [0.001, 0.01, 0.05, 0.1, 0.25, 0.5], "depth": [2,3,4]}

# grid_search_result = model.grid_search(grid, X=X_train, y=y_train)
# dep = grid_search_result["params"]["depth"]
# lr = grid_search_result["params"]["learning_rate"]
# print(grid_search_result["params"])
######################################################################

model = CatBoostClassifier(iterations=1000, learning_rate=lr, \
                            depth=dep, cat_features=categorical_features, \
                                loss_function="CrossEntropy", logging_level="Silent")

model.fit(X_train_scaled, y_train);

In [13]:
train_predictions = model.predict(X_train_scaled)
print("Score on training set: ", accuracy_score(y_train, train_predictions))

Score on training set:  0.9450056116722784


In [14]:
# Output submission csv as detailed in Kaggle.
test_predictions = model.predict(X_test_scaled)
solution = pd.DataFrame( { "PassengerId": df_test.PassengerId, "Survived": test_predictions } )
solution.to_csv("submission\submissionCatboost.csv", index = False)